# Exploratory Data Analysis(EDA) 

In [1]:
# import the libraries
import pandas as pd
import numpy as np

In [2]:
# import utility modules via Url
import httpimport

address = "https://raw.githubusercontent.com/Gkchandora/Breast_Cancer_Prediction/main/Model_Building_Steps"

with httpimport.remote_repo(["utility_modules"], address):
    from utility_modules import *

[-] 'httpimport.INSECURE' is not set! Aborting...


Exception: ignored